# Run SSA and weighted PCA on center-out (red and yellow, long delay) reaching data. Plot projections in pca and sca dimensions
- first generate Balboa figures, then Alex.
- Use default orthogonality and sparsity penalties for SCA

## Import packages

In [ ]:
import numpy as np
import numpy.random as npr
from scipy import io
import seaborn as sns
import time

import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt

import sys
sys.path.append('/Users/sherryan/glaserlab/sca_analysis_parent/ssa')

from ssa.models import fit_ssa, weighted_pca
from ssa.util import get_sample_weights
from plotly.subplots import make_subplots

from sklearn.decomposition import FastICA

# add appropriate directories to search path
# import sys
# sys.path.append('/Users/andrew/Documents/Projects/Churchland/Sparsity/code/andrewPython/')

# from centerOutReaching_utils import calculateEpochOccupancy
# from centerOutReaching_utils import calculateOccDispersion
# from centerOutReaching_utils import calculateAI
# from centerOutReaching_utils import calculateChanceAI
# from parallelFunctions import bootstrapNeurons_SCA_PCA


## Load Data

In [ ]:
# which monkey are we working with?
monkName = 'Balboa'

In [ ]:
# folder with rates
load_folder='/Users/sherryan/glaserlab/sca_analysis_parent/sca_analysis/'

# load data
data=io.loadmat(load_folder + monkName + '_outAndBack_redYellowConds_rawRates.mat')

# pull out the psths
# data is a C x N x T tensor 
data_array=data['data']

## Preprocess data


In [ ]:
#Downsample data (using a factor of 10 here)
data_downsamp=data_array[:,:,np.arange(0,data_array.shape[2],10)]

# pull out some useful numbers
numConds,numN,trlDur = np.shape(data_downsamp)

#Concatenate all the conditions (so the matrix is size N x TC instead of C x N x T)
data_concat=data_downsamp.swapaxes(0,1).reshape([data_downsamp.shape[1],data_downsamp.shape[0]*data_downsamp.shape[2]])

#fr range
fr_range=np.ptp(data_concat,axis=1)[:,None]

# make a time mask
timeMask = np.tile(np.arange(trlDur),(1,numConds)).T.flatten()

# define the times we want to use for ssa/pca
# target on: 20
# move on:   77
# return:    200
trainTimes = np.arange(20,230)

# define a 'training mask' for convenience 
trainMask = np.in1d(timeMask,trainTimes)

#Subtract cross-condition mean
data_scm=data_downsamp-np.mean(data_downsamp,axis=0)[None,:,:]

#Concatenate all the conditions (so the matrix is size N x TC instead of C x N x T)
data_scm_concat=data_scm.swapaxes(0,1).reshape([data_scm.shape[1],data_scm.shape[0]*data_scm.shape[2]])

#Soft normalize (divide each neuron by its fr range + 5)
data_scm_norm=data_scm_concat/(fr_range+5)

# mean-center the data
dMean = np.tile(np.mean(data_scm_norm,axis=1)[:,np.newaxis],(1,data_scm_norm.shape[1]))
data_mc = data_scm_norm - dMean

# rename the data for convenience
#Note that model requires (T x N) input rather than (N x T), which is why there are transposes below
fit_data=np.copy(data_mc.T)

# how much to weight each timestep (used by)
sample_weights=get_sample_weights(fit_data)

# number of dimensions to find
R_est=24

# Whether or not to impose hard orthogonality constraint
hardOrthFlag = False


## Fit SCA

In [ ]:
# fit model
# model,ssa_latent, x_pred,losses=fit_ssa(X=fit_data[trainMask,:],sample_weight = sample_weights[trainMask],
#                                 R=R_est,orth = hardOrthFlag)
model,ssa_latent, x_pred,losses=fit_ssa(X=fit_data[trainMask,:],sample_weight = sample_weights[trainMask],
                                R=R_est,lam_orthog=0,orth = hardOrthFlag)
# grab weights
ssaW = model.fc1.weight.detach().numpy()

# project all of the data into the ssa dimensions
ssa_latent = fit_data @ ssaW.T

# calculate reconstruction R2
X_hat = x_pred.detach().numpy()
X = fit_data[trainMask,:]


### Fit ICA

In [ ]:
transformer = FastICA(n_components = R_est, whiten = 'unit-variance',max_iter = 500)
icaModel = transformer.fit(fit_data[trainMask,:])
icaLatents = icaModel.fit_transform(fit_data[trainMask,:])
icaW = icaModel.components_

# get ICA projections
ica_latent = fit_data @ icaW.T


### define some plotting colors
 - ssa is purple -> light purple
 - I used https://davidjohnstone.net/lch-lab-colour-gradient-picker

In [ ]:
# define ssa colors
sca_cMap  = ['#5e0044', '#6f144e', '#812858', '#933c62', '#a5506d', '#b76477', '#c97881', '#db8c8c']
ica_cMap  = ['#6effc0', '#62ebb2', '#57d7a5', '#4bc398', '#40af8a', '#349b7d', '#298770', '#1e7363']

## Plot latents

### Plot when reordering by time of maximal influence

In [ ]:
# calculate across-condition variance of each projection as a function of time

# threshold for determining when a given dimension is occupied. (relative to max occupancy)
occThresh_fract = 0.1

# reshape both latents to be size T x C x K 
rs_sca_latent = np.reshape(ssa_latent,(-1,8,R_est),order = 'F')

# calculate across condition variance
sca_var = np.var(rs_sca_latent,axis = 1)

# calculate max occupancy for each dimension
maxOcc_sca = np.max(sca_var, axis=0)

# set occupancy threshold
occThresh = np.max(maxOcc_sca) * occThresh_fract

# initialize a vector for sorting the dimensions and move unoccupied dimensions to the end of the list
sca_order = np.zeros(R_est)

# detect low occupancy dimensions
lowOccDims = np.argwhere(maxOcc_sca < occThresh)[:,0]
highOccDims = np.argwhere(maxOcc_sca >= occThresh)[:,0]
numLowOcc = lowOccDims.shape[0]
numHighOcc = highOccDims.shape[0]

# move them to the end of the list
sca_order[-numLowOcc:] = lowOccDims

# grab the rest of the dimensions and sort them by time of peak occupancy
sca_var = sca_var[:,highOccDims]

# find peak occupancy of each dimension
pkIdx = np.argmax(sca_var,axis = 0)

# define plotting order of the high occupancy dimensions and add to list
highOrder = np.argsort(pkIdx)
sca_order[:numHighOcc] = highOccDims[highOrder]
sca_order = np.copy(sca_order.astype(int))

# resort ssa_latents by time of maximum occupancy
rs_sca_latent = rs_sca_latent[:,:,sca_order]


# # do the same for ica

# # reshape both latents to be size T x C x K
# rs_ica_latent = np.reshape(ica_latent,(-1,8,R_est),order = 'F')

# # calculate across condition variance
# ica_var = np.var(rs_ica_latent,axis = 1)

# # calculate max occupancy for each dimension
# maxOcc_ica = np.max(ica_var, axis=0)

# # set occupancy threshold
# occThresh = np.max(maxOcc_ica) * occThresh_fract

# # initialize a vector for sorting the dimensions and move unoccupied dimensions to the end of the list
# ica_order = np.zeros(R_est)

# # detect low occupancy dimensions
# lowOccDims = np.argwhere(maxOcc_ica < occThresh)[:,0]
# highOccDims = np.argwhere(maxOcc_ica >= occThresh)[:,0]
# numLowOcc = lowOccDims.shape[0]
# numHighOcc = highOccDims.shape[0]

# # move them to the end of the list
# if numLowOcc > 0:
#     ica_order[-numLowOcc:] = lowOccDims

# # grab the rest of the dimensions and sort them by time of peak occupancy
# ica_var = ica_var[:,highOccDims]

# # find peak occupancy of each dimension
# pkIdx = np.argmax(ica_var,axis = 0)

# # define plotting order of the high occupancy dimensions and add to list
# highOrder = np.argsort(pkIdx)
# ica_order[:numHighOcc] = highOccDims[highOrder]
# ica_order = np.copy(ica_order.astype(int))

# # resort ssa_latents by time of maximum occupancy
# rs_ica_latent = rs_ica_latent[:,:,ica_order]


In [ ]:
## Define the directory for our new figures
figDir = '/Users/andrew/Documents/Projects/Churchland/Sparsity/figures/centerOutReaching/defaultSCAParameters/'

#### SCA

In [ ]:
# range for y axis
tgt_idx=20
move_idx=77
ret_idx=200
yRange = [-2,1.7]

# number of columns and row
numRows = int(R_est/2)

fig = make_subplots(rows=numRows,cols = 2,shared_xaxes = True,vertical_spacing = 0)

for ii in range(R_est):

    rIdx = (ii%numRows)+1
    cIdx = int(np.floor(ii/numRows) +1)

    for jj in range(numConds):
        latTrace = go.Scatter(y = rs_sca_latent[:,jj,ii], line = go.scatter.Line(color = sca_cMap[jj],width = 1),showlegend = False)
        fig.add_trace(latTrace,row = rIdx,col=cIdx)

    # mark task times
    fig.add_vline(x = tgt_idx,row = rIdx,col = cIdx, line_color = 'black')
    fig.add_vline(x = move_idx,row = rIdx,col = cIdx, line_color = 'black')
    fig.add_vline(x = ret_idx,row = rIdx,col = cIdx, line_color = 'black')

            #
    # add a vertical line for scale
    scaleLine = go.Scatter(x = [0,0],y = [-1,1],showlegend = False,mode = 'lines',
                            line = go.scatter.Line(color = 'black',width = 2))
    fig.add_trace(scaleLine,row = rIdx,col = cIdx)


fig.update_layout(height = 1300,width =800,title = 'SCA with orthogonality',title_font_color = 'black',
                  paper_bgcolor = 'white',
                  plot_bgcolor = 'white')
fig.update_yaxes(showgrid = False,zeroline = False,visible = False,range = yRange)
fig.update_xaxes(color = 'black',showgrid = False,zeroline = False,visible = False)
fig.update_xaxes(color = 'black',showgrid = False,zeroline = False,
                 ticks = 'outside',tickvals = [0,50],ticktext = ['0','500'],visible = True,row = numRows,col = 2)


# save
fig.write_image(monkName + 'SCA_orthog' + str(R_est) + 'dims.pdf')
fig.show()


In [ ]:
# range for y axis
tgt_idx=20
move_idx=77
ret_idx=200
yRange = [-2,1.7]

# number of columns and row
numRows = int(R_est/2)

fig = make_subplots(rows=numRows,cols = 2,shared_xaxes = True,vertical_spacing = 0)

for ii in range(R_est):

    rIdx = (ii%numRows)+1
    cIdx = int(np.floor(ii/numRows) +1)

    for jj in range(numConds):
        latTrace = go.Scatter(y = rs_sca_latent[:,jj,ii], line = go.scatter.Line(color = sca_cMap[jj],width = 1),showlegend = False)
        fig.add_trace(latTrace,row = rIdx,col=cIdx)

    # mark task times
    fig.add_vline(x = tgt_idx,row = rIdx,col = cIdx, line_color = 'black')
    fig.add_vline(x = move_idx,row = rIdx,col = cIdx, line_color = 'black')
    fig.add_vline(x = ret_idx,row = rIdx,col = cIdx, line_color = 'black')

            #
    # add a vertical line for scale
    scaleLine = go.Scatter(x = [0,0],y = [-1,1],showlegend = False,mode = 'lines',
                            line = go.scatter.Line(color = 'black',width = 2))
    fig.add_trace(scaleLine,row = rIdx,col = cIdx)


fig.update_layout(height = 1300,width =800,title = 'SCA with no orthogonality',title_font_color = 'black',
                  paper_bgcolor = 'white',
                  plot_bgcolor = 'white')
fig.update_yaxes(showgrid = False,zeroline = False,visible = False,range = yRange)
fig.update_xaxes(color = 'black',showgrid = False,zeroline = False,visible = False)
fig.update_xaxes(color = 'black',showgrid = False,zeroline = False,
                 ticks = 'outside',tickvals = [0,50],ticktext = ['0','500'],visible = True,row = numRows,col = 2)


# save
fig.write_image(monkName + 'SCA_wo_orthog' + str(R_est) + 'dims.pdf')
fig.show()


### ICA

In [ ]:
# range for y axis
yRange = [-9.5,9.5]

# number of columns and row
numRows = int(R_est/2)

fig = make_subplots(rows=numRows,cols = 2,shared_xaxes = True,vertical_spacing = 0)

for ii in range(R_est):

    rIdx = (ii%numRows)+1
    cIdx = int(np.floor(ii/numRows) +1)

    for jj in range(numConds):
        latTrace = go.Scatter(y = rs_ica_latent[:,jj,ii], line = go.scatter.Line(color = ica_cMap[jj],width = 2.5),showlegend = False)
        fig.add_trace(latTrace,row = rIdx,col=cIdx)

    # mark task times
    fig.add_vline(x = tgt_idx,row = rIdx,col = cIdx, line_color = 'black')
    fig.add_vline(x = move_idx,row = rIdx,col = cIdx, line_color = 'black')
    fig.add_vline(x = ret_idx,row = rIdx,col = cIdx, line_color = 'black')

            #
    # add a vertical line for scale
    scaleLine = go.Scatter(x = [0,0],y = [-5,5],showlegend = False,mode = 'lines',
                            line = go.scatter.Line(color = 'black',width = 5))
    fig.add_trace(scaleLine,row = rIdx,col = cIdx)


fig.update_layout(height = 1300,width =800,title = 'ICA ' + monkName,title_font_color = 'black',
                  paper_bgcolor = 'white',
                  plot_bgcolor = 'white')
fig.update_yaxes(showgrid = False,zeroline = False,visible = False,range = yRange)
fig.update_xaxes(color = 'black',showgrid = False,zeroline = False,visible = False)
fig.update_xaxes(color = 'black',showgrid = False,zeroline = False,
                 ticks = 'outside',tickvals = [0,50],ticktext = ['0','500'],visible = True,row = numRows,col = 2)


# save
fig.write_image(figDir + monkName + 'ICA_' + str(R_est) + 'dims.pdf')
fig.show()

### Will ICA or SCA 'oversparsify' if a user asks for too many dimensions?
    - for each set of dimensions, calculate average occupancy/dimension.
    - SCA should stop finding 'real' dimensions at some point
    - ICA will continue to split as much as it can (no orthogonality penalty)

In [ ]:
# number of dimensions we want
numDims = list(np.arange(5,100,5))

# initialize lists to hold all latents
ica_latents = list()
sca_latents = list()

# save weights too
ica_weights = list()
sca_weights = list()

# cycle through dimensions
for ii in numDims:

    ## sca first ##
    # fit model
    model,sca_latent, x_pred,losses=fit_sca(X=fit_data[trainMask,:],sample_weight = sample_weights[trainMask],
                                    R=ii,orth = hardOrthFlag)

    # grab and save latents and weights
    scaW = model.fc1.weight.detach().numpy()
    sca_latents.append(fit_data @ scaW.T)
    sca_weights.append(scaW.T)

    ## ica ##
    transformer = FastICA(n_components = ii, whiten = 'unit-variance',max_iter = 500)
    icaModel = transformer.fit(fit_data[trainMask,:])
    icaLatents = icaModel.fit_transform(fit_data[trainMask,:])
    icaW = icaModel.components_

    #save latents and weights
    ica_latents.append(fit_data @ icaW.T)
    ica_weights.append(icaW.T)



In [ ]:
# save these latents
saveDir = '/Users/andrew/Documents/Projects/Churchland/Sparsity/data/reaching/sca_vs_ica_latents/'
np.save(saveDir + monkName + '_scaIcaLatents.npy',{'scaLatents': sca_latents,'icaLatents': ica_latents,'numDims': numDims, 'icaW': ica_weights, 'scaW': sca_weights, 'fit_data': fit_data})



In [ ]:
# or load the latents
saveDir = '/Users/andrew/Documents/Projects/Churchland/Sparsity/data/reaching/sca_vs_ica_latents/'
data = np.load(saveDir + monkName + '_scaIcaLatents.npy',allow_pickle=True)
data = data.ravel()[0]

# pull out the sca and ica weights
scaW = data['scaW']
icaW = data['icaW']

numDims = list(np.arange(5,100,5))

In [ ]:
# for each set of latents, calculate fraction of 'occupied' dimensions (dimensions with occupancy > some threshold)
# also calculate occupancy/dimension

# vectors to hold results
avgOcc_ica = np.zeros(len(numDims))
avgOcc_sca = np.zeros(len(numDims))

numOccDims_ica = np.zeros(len(numDims))
numOccDims_sca = np.zeros(len(numDims))

# threshold for saying a dimension is 'occupied'
# define a dimension as 'occupied' if the max occupancy is > 0.3 of the max
occThreshold = 0.30

# cycle through sets of dimensions
for ii in range(1,len(numDims)):

    # start with sca latents
    sca_lat = sca_latents[ii]

    # reshape to be T x C x K
    sca_lat_rs = sca_lat.reshape(-1,8,numDims[ii],order = 'F')

    # calculate occupancy for each dimension
    # first normalize weights so we can compare across ICA and SCA
    W_sca = sca_weights[ii]
    W_sca = W_sca / np.linalg.norm(W_sca,axis = 0)
    sca_lat_norm = fit_data @ W_sca
    sca_lat_norm_rs = sca_lat_norm.reshape(-1,8, numDims[ii], order='F')
    dimOcc_sca = np.sum(np.var(sca_lat_norm_rs, axis=1), axis=0)

    # calculate maximum occupancy for each dimension
    maxOcc_sca = np.max(np.var(sca_lat_rs,axis=1), axis=0)

    # get the threshold for occupancy
    occThresh_sca = np.max(maxOcc_sca) * occThreshold

    # get fraction of dimensions that are occupied
    numOccDims_sca[ii] = np.copy(np.sum(maxOcc_sca >= occThresh_sca)/numDims[ii])

    # calculate average occupancy
    avgOcc_sca[ii] = np.copy(np.mean(dimOcc_sca))

    # now the ica latents
    ica_lat = ica_latents[ii]

    # reshape to be T x C x K
    ica_lat_rs = ica_lat.reshape(-1,8,numDims[ii],order = 'F')

    # calculate occupancy for each dimension after normalizing
    W_ica = ica_weights[ii]
    W_ica = W_ica / np.linalg.norm(W_ica,axis = 0)
    ica_lat_norm = fit_data @ W_ica
    ica_lat_norm_rs = ica_lat_norm.reshape(-1,8, numDims[ii], order='F')
    dimOcc_ica = np.sum(np.var(ica_lat_norm_rs, axis=1), axis=0)

    # calculate maximum occupancy for each dimension
    maxOcc_ica = np.max(np.var(ica_lat_rs,axis=1), axis=0)

    # get the threshold for occupancy
    occThresh = np.max(maxOcc_ica) * occThreshold

    # get fraction of dimensions that are occupied
    numOccDims_ica[ii] = np.copy(np.sum(maxOcc_ica >= occThresh)/numDims[ii])

    # calculate average occupancy
    avgOcc_ica[ii] = np.copy(np.mean(dimOcc_ica))



In [ ]:
# plot results
fig = make_subplots(rows = 1, cols = 2, horizontal_spacing = 0.05,subplot_titles = ['mean occupancy','fraction of occupied dimensions'])

fig.add_trace(go.Scatter(x = numDims[1:],y = avgOcc_ica[1:], line = go.scatter.Line(color = 'grey',width = 2.5 ),name = 'ica', mode='lines'),row = 1, col=1)
fig.add_trace(go.Scatter(x = numDims[1:],y = avgOcc_sca[1:], line = go.scatter.Line(color = 'black',width = 2.5 ),name = 'sca', mode='lines'),row = 1, col=1)

fig.add_trace(go.Scatter(x = numDims[1:],y = numOccDims_ica[1:], line = go.scatter.Line(color = 'grey',width = 2.5 ),name = 'ica', mode='lines'),row = 1, col=2)
fig.add_trace(go.Scatter(x = numDims[1:],y = numOccDims_sca[1:], line = go.scatter.Line(color = 'black',width = 2.5 ),name = 'sca', mode='lines'),row = 1, col=2)


# clean up figure
fig.update_layout(height =300,width =600,title = 'SCA vs. ICA ' + monkName,title_font_color = 'black',
                  paper_bgcolor = 'white',
                  plot_bgcolor = 'white', font=dict(family='Avenir',color='black'))

fig.update_yaxes(showgrid = False,zeroline = False, ticks = 'outside', color = 'black',
                 showline = True, linewidth=1.5, tickwidth=1.5, linecolor = 'black',
                 tickvals = [0,20], range=[0,20],row = 1, col = 1)
fig.update_xaxes(showgrid = False,zeroline = False, ticks = 'outside', color = 'black',
                 showline = True, linewidth=1.5, tickwidth=1.5, linecolor = 'black',
                 tickvals = [0,50,100], range=[0,100],row = 1, col = 1)

fig.update_yaxes(showgrid = False,zeroline = False, ticks = 'outside', color = 'black',
                 showline = True, linewidth=1.5, tickwidth=1.5, linecolor = 'black',
                 tickvals = [0,0.5,1], range=[0,1],row = 1, col = 2)

fig.update_xaxes(showgrid = False,zeroline = False, ticks = 'outside', color = 'black',
                 showline = True, linewidth=1.5, tickwidth=1.5, linecolor = 'black',
                 tickvals = [0,50,100], range=[0,100],row = 1, col = 2)

# save figure
fig.write_image(figDir + monkName + 'icaVsSca_quant.pdf')
fig.show()


In [ ]:


# range for y axis
yRange = [-1.8,1.2]

fig = make_subplots(rows=R_est,cols = 1,shared_xaxes = True,vertical_spacing = 0)

for ii in range(R_est):

    for jj in range(numConds):
        latTrace = go.Scatter(y = rs_sca_latent[:,jj,ii], line = go.scatter.Line(color = sca_cMap[jj],width = 2.5),showlegend = False)
        fig.add_trace(latTrace,row = ii+1,col=1)
        if ii == (R_est-1):
            fig.add_vline(x = tgt_idx,row = ii+1,col = 1, line_color = 'black')
            fig.add_vline(x = move_idx,row = ii+1,col = 1, line_color = 'black')
            fig.add_vline(x = ret_idx,row = ii+1,col = 1, line_color = 'black')

            #
    # add a vertical line for scale
    scaleLine = go.Scatter(x = [0,0],y = [-1,1],showlegend = False,mode = 'lines',
                            line = go.scatter.Line(color = 'black',width = 5))
    fig.add_trace(scaleLine,row = ii+1,col = 1)


fig.update_layout(height = 1000,width =600,title = 'SCA ' + monkName,title_font_color = 'black',
                  paper_bgcolor = 'white',
                  plot_bgcolor = 'white')
fig.update_yaxes(showgrid = False,zeroline = False,visible = False,range = yRange)
fig.update_xaxes(color = 'black',showgrid = False,zeroline = False,visible = False)
fig.update_xaxes(color = 'black',showgrid = False,zeroline = False,
                 ticks = 'outside',tickvals = [0,50],ticktext = ['0','500'],visible = True,row = R_est,col = 1)


# save
#fig.write_image(figDir + monkName + 'SCA_' + str(R_est) + 'dims.pdf')
#fig.show()


In [ ]:
#  measure the angle between each dimensions within each set of ICA and SCA weights

# initialize list to hold angles
scaTheta = list()
icaTheta = list()

# number of sets
numDimSets = len(scaW)

# cycle through sets
for ii in range(numDimSets):

    # number of dimensions in this set
    numDims = scaW[ii].shape[1]

    # initialize lists to hold angles
    tempSCATheta = list()
    tempICATheta = list()

    # normalize all dimensions to have magnitude 1
    sca_norm = scaW[ii] / np.linalg.norm(scaW[ii],axis=0)
    ica_norm = icaW[ii] / np.linalg.norm(icaW[ii],axis=0)

    # cycle through dimensions
    for jj in range(numDims):
        for kk in range(numDims):

            # no duplicate angles
            if kk > jj:

                # pull out sca weights
                sca1 = sca_norm[:,jj]
                sca2 = sca_norm[:,kk]

                # ica weights
                ica1 = ica_norm[:,jj]
                ica2 = ica_norm[:,kk]

                # measure angle between sca vectors (in degrees)
                scaT = np.arccos(np.dot(sca1,sca2)) * (180/np.pi)
                icaT = np.arccos(np.dot(ica1,ica2)) * (180/np.pi)

                # adjust for angles greater than 90 deg
                if scaT > 90:
                    scaT = 180 - scaT
                if icaT > 90:
                    icaT = 180 - icaT

                # add to the growing list
                tempSCATheta.append(scaT)
                tempICATheta.append(icaT)

    # add to larger list
    scaTheta.append(tempSCATheta)
    icaTheta.append(tempICATheta)

# find the 10th percentile for each set of angles
pTile = 0.01

# sca first
sca_pTiles = [np.percentile(x,pTile) for x in scaTheta]

# ica
ica_pTiles = [np.percentile(x,pTile) for x in icaTheta]



In [ ]:
a = sum([len(x) for x in scaTheta])
print(a)
b = sum([len(x) for x in icaTheta])
print(b)

In [ ]:
plt.plot(sca_pTiles,'b');
plt.plot(ica_pTiles,'r');
plt.ylim([45,90]);

In [ ]:
# plot each angle

# figure directory
figDir = '/Users/andrew/Documents/Projects/Churchland/Sparsity/figures/centerOutReaching/defaultSCAParameters/'

# accidentally redefined this variable above
numDims = list(np.arange(5,100,5))

# offset between SCA and ICA
offset = 1

# marker alpha
alpha = 0.1

# plot results
fig = make_subplots(rows = 1, cols = 1,subplot_titles = ['average angle between dimensions'])

# cycle through sets
for ii,jj in enumerate(numDims):

    if ii > 0:

        # plot every 5th angle difference to cut down on image size
        allDiffs_sca = np.array(scaTheta[ii])
        allDiffs_ica = np.array(icaTheta[ii])
        idx = np.arange(0,allDiffs_sca.shape[0],8)
        sca = allDiffs_sca[idx]
        ica = allDiffs_ica[idx]


        # number of angles
        numAngles = sca.shape[0]

        fig.add_trace(go.Scatter(x = np.zeros(numAngles) + jj - offset,y = ica, mode='markers', showlegend=False, marker_color='rgba(100,100,100,0.1)') ,row = 1, col=1)
        fig.add_trace(go.Scatter(x = np.zeros(numAngles) + jj + offset,y = sca, mode='markers', showlegend=False, marker_color='rgba(0,0,0,0.1)'),row = 1, col=1)


# clean up figure
fig.update_layout(height =300,width =400,title = 'SCA vs. ICA ' + monkName,title_font_color = 'black',
                  paper_bgcolor = 'white',
                  plot_bgcolor = 'white', font=dict(family='Avenir',color='black'))

fig.update_traces(mode='markers', marker_line_width=None, marker_size=4)

fig.update_yaxes(showgrid = False,zeroline = False, ticks = 'outside', color = 'black',
                 showline = True, linewidth=1.5, tickwidth=1.5, linecolor = 'black',
                 tickvals = [45,90], range=[45,90],row = 1, col = 1)
fig.update_xaxes(showgrid = False,zeroline = False, ticks = 'outside', color = 'black',
                 showline = True, linewidth=1.5, tickwidth=1.5, linecolor = 'black',
                 tickvals = [0,50,100], range=[0,105],row = 1, col = 1)

# save figure
#fig.write_image(figDir + monkName + 'icaVsSca_dimensionAngleDiff_downsample.pdf')
#fig.show()
